# LIVE 04 - Modelos na vida real

Esse tutorial original foi feito pela GRANDE Alexis Cook (que muito me ensinou nos Nanodegrees da Udacity). Traduzi grande parte e mudei algumas coisas, mas a linha geral se manteve.

O original está aqui: https://www.kaggle.com/alexisbcook/data-leakage

Neste tutorial você vai aprender o que é **data leakage** e como evitá-lo! Na nossa aula vimos as formas mais "nutella" de data leakage, que são MUITO importantes, mas geralmente mais fáceis de identificar (mesmo sendo difícil!!)

Veremos que mesmo algumas das nossas variáveis utilizadas no modelo preditivo podem trazer informações do futuro!! Esse tipo é o mais difícil de identificar e esperamos que fique mais claro depois da apresentação.


# Introdução

**Data leakage** acontece quando os dados de treino contém algum tipo de informação sobre a variável dependente (nosso target de predição) que não vai estar disponível quando o modelo começar as predições reais em produção. Isso leva a uma falsa impressão que o modelo tem uma ótima performance, quando, na verdade, ele teve um vazamento de informação que facilita a predição, mas quando chegar na produção.... vai ficar uma porcaria.

A Alexis usa uma classificação bem legal de tipos de vazamento de dados: **target leakage** e **contaminação treino-teste.**

### Target leakage

**Target leakage** acontace quando suas variáveis independentes incluem dados que não vão estar disponíveis na hora em que o modelo vai realizar as predições. Long story short, você tem que analisar se os dados que você usou no treinamento não são uma "antecipação" do futuro. Desse modo, é importante pensar nesse tipo de vazamento como algo _cronológico_. 

Um exemplo pra ver na prática!! Vamos predizer quem vai ter ou não pneumonia a partir dos dados de: idade, peso, homem e tomou antibiotico:

| tem_pneumonia | idd |  peso  | homem |     tomou_antibiotico    | ... |
|:-------------:|:---:|:------:|:-----:|:------------------------:|-----|
|     False     |  65 |   100  | False |           False          | ... |
|     False     |  72 |   130  |  True |           False          | ... |
|      True     |  58 |   100  | False |           True           | ... |

Pessoas tomam antibiótico *DEPOIS* de pegar pneumonia. Os dados brutos mostra uma grande correlação entre as colunas target e tomou_antibiotico, mas essa última normalmente é alterada _DEPOIS_ que a pneumonia é detectada. Aqui está o vazamento!!

O modelo, espertão que é, logo vai perceber a correlação e nem vai dar bola pras outras variáveis! Se o cara não tomou antibiótico, não teve pneumonia!!! Seu modelo vai ficar *BALA*, mas é um ledo engano, porque quando você começar a prever os próximos casos, a variável *tomou_antibiotico* não vai estar com os valores setados (ou todos pra false) e você fica na mão.

Nesse caso não há cross validation ou hould out que ajude! Os seus dados já estão contaminados!

Pra evitar isso, devemos REMOVER as variáveis que são criadas depois do momento de predição:

![tut7_leakydata](https://i.imgur.com/y7hfTYe.png)

### Contaminação Treino-Teste (validação)

Esse é o caso que vimos na aula! No pré-processamento devemos tomar o cuidado de separar os dados *ANTES* para não haver esse tipo de contaminação!

Em termos práticos, temos que fazer a separação antes! No scikit learn tem o conhecido [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) que já faz isso pra gente!

Lembrando que, a partir dessa separação, vamos usar a parte de treino + cross validation e deixar o teste *PRA LÁ*!


# Exemplo

Nesse exemplo, vamos aprender a identificar e remover as variáveis que possam ter vazamento para a variável dependente (target leakage).

Vamos usar uma base de dados de aplicações de cartão de crédito. Vamos tentar prever se as aplicações para cartões de crédito feito pelas pessoas foi aceito pela instituição financeira ou não (risco de crédito)! Veremos se pode haver alguma variável independente que tenha informações da variável que queremos prever.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# Ler os dados
data = pd.read_csv('./data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

data.head()

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,True,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,True,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,True,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,True,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,True,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


In [2]:
# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print(f"Número de linhas {X.shape[0]}")
X.head()

Número de linhas 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Como temos uma base de dados pequena, vamos usar validação cruzada pra garantir medidas precisas da qualidade do modelo.

In [3]:
# Importando pacotes
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Como não tem pré-processamento, nem precisa de pipeline, mas vamos manter as boas práticas
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

In [4]:
print(f"Acurácia no Cross-validation: {cv_scores.mean()}")

Acurácia no Cross-validation: 0.9802915082382764


Ficou bom demais, quando é assim *sempre desconfie*!! O principal erro dos iniciantes é comemorar os resultados, o bom cientista de dados desconfia de tudo!

Vamos analisar as variáveis pra ver se existe algum problema!!

Uma descrição dos dados:

 - **`card`**: True se a aplicação do cartão foi aceita, False caso contrário
 - **`reports`**: Número de relatório depreciativos (tradução literal, quem é de banco aí pra traduzir??)
 - **`age`**: Idade
 - **`income`**: Salário anual (dividido por 10k)
 - **`share`**: Gasto em cartão dividido pelo salário anual (razão gasto cartão/salário)
 - **`expenditure`**: Gasto médio mensal em cartão de crédito
 - **`owner`**: True se tem um imóvel, False se aluga
 - **`selfempl`**: True se tem emprego próprio, False se não
 - **`dependents`**: Número de dependentes
 - **`months`**: Meses morando no endereço atual
 - **`majorcards`**: Quanto cartões possui (dos grandes!)
 - **`active`**: Número de contas de crédito ativas

Algumas variáveis são bem suspeitas!!! Por exemplo, **`expenditure`** (gasto médio mensal), é no cartão atual ou nos utilizados antes da aplicação?

Uma comparação entre os dados pode ajudar!

In [5]:
# Gastos de quem possui cartão
gastos_possuem_cartao = X.expenditure[y]

# Gastos de quem não possui cartão
gastos_nao_possuem_cartao = X.expenditure[~y]

print('Fração dos que não receberam cartão e não possuem gastos: %.2f' \
      %((gastos_nao_possuem_cartao == 0).mean()))
print('Fração dos que receberam cartão e não possuem gastos: %.2f' \
      %(( gastos_possuem_cartao == 0).mean()))

Fração dos que não receberam cartão e não possuem gastos: 1.00
Fração dos que receberam cartão e não possuem gastos: 0.02


Ou seja, todo mundo que não recebeu o cartão, também não tem gastos! Enquanto somente 2%  dos que receberam o cartão não tem gastos. Assim fica fácil! Se os gastos futuros já estão aqui disponíveis, o modelo que não é bobo nem nada, aprendeu rapidindo e nem olhou pro resto, acerta quase tudo!

Como a variável **`share`** também é determinada indiretamente pelos gastos, também deveria ser excluída.

As variáveis **`active`** e **`majorcards`** não dão muita certeza, como não temos mais informações, melhor não arriscar e removê-las também. No mundo real nós teríamos que investigar de onde elas surgiram e como foram coletadas!

Vamos rodar o modelo de novo sem as variáveis com potencial de vazamento:

In [6]:
# Remvoer as variáveis que podem ter vazamento
vazamento_potencial = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(vazamento_potencial, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

In [7]:
print(f"Acurácia no Cross-validation: {cv_scores.mean()}")

Acurácia no Cross-validation: 0.8316856780735108


Não fique triste!! A acurácia não é muito boa, mas é melhor você saber a realidade do modelo do que ser iludido por uma métrica mais bonita, mas que vai falhar miseravelmente em produção! 


# Conclusão

Data leakage pode incorrer em erros multimilionários em aplicações de data science! Cuidado ao separar o treino, validação e teste. Como vimos nesse exemplo, sempre verifique o potencial de ter também target leakage!


# A partir daqui?

O tutorial da nossa rainha Alexis Cook aponta para um exercício para saber se vocês entenderam o que é data leakage, corram lá pra fazer **[o exercício](https://www.kaggle.com/kernels/fork/3370270)**